In [1]:
import torch
import random
import pandas as pd
import json
import numpy as np
import torch.backends.cudnn as cudnn

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
# seed 결과가달라짐 3~4%

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print('device:', device)

device: cuda:0


In [3]:
print('-'*10)
print('Data Loading Start!!')
print('-'*10)

## dataset class
path = "/data/ephemeral/home/data/aug_data_x10.jsonl"

with open(path) as f:
    data = [json.loads(line) for line in f]
data = pd.DataFrame(data)
print('origin_data:', data.shape)


----------
Data Loading Start!!
----------
origin_data: (42720, 5)


In [4]:
data.columns

Index(['docid', 'question', 'content', 'src', 'new_domains'], dtype='object')

In [6]:
data['new_domains'].unique()

array(['human_aging', 'medical_genetics', 'high_school_biology',
       'college_chemistry', 'college_physics', 'conceptual_physics',
       'global_facts', 'None', 'unknown', 'computer_security',
       'high_school_chemistry', 'anatomy', 'nutrition', 'human_sexuality',
       'astronomy', 'high_school_computer_science', 'virology',
       'electrical_engineering', 'college_medicine', 'college_biology',
       'college_computer_science', 'human_aging, nutrition',
       'high_school_physics', 'geology', 'art', 'college_science',
       'safety', 'civil_engineering', 'new_technology_in_industry',
       'engineering', 'data_visualization',
       'human_sexuality, medical_genetics', 'environmental_science',
       'music_performance', 'logistics', 'None, medical_genetics',
       'astronomy, college_physics', 'construction_tools',
       'astronomy, college_biology', 'astronomy, conceptual_physics',
       'anatomy, college_chemistry', 'safety_education',
       'investigation_records'

In [5]:
data.shape

(42720, 5)

In [6]:
data[data['docid']=='c73343e8-395d-40d0-854a-529d11c4e194']

,docid,question,content,src,new_domains
33360,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들은 어떤 대상을 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33361,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들은 어떤 객체를 연구하나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33362,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자의 연구 방법에는 어떤 공통점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33363,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치의 예시로 어떤 것들이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33364,c73343e8-395d-40d0-854a-529d11c4e194,천문학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33365,c73343e8-395d-40d0-854a-529d11c4e194,생물학자들이 사용하는 장치는 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33366,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치를 사용하면 어떤 이점이 있나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33367,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자는 어떤 분야의 연구자들인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33368,c73343e8-395d-40d0-854a-529d11c4e194,천문학자와 생물학자가 연구하는 대상은 어떻게 다르나요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"
33369,c73343e8-395d-40d0-854a-529d11c4e194,광학 장치가 발견에 미치는 영향은 무엇인가요?,천문학자와 생물학자는 과학의 다른 영역을 연구합니다. 천문학자들은 하늘에 있는 매우...,ko_ai2_arc__ARC_Challenge__train,"astronomy, college_biology"


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# 데이터셋 정의
class PositivePairDataset(Dataset):
    def __init__(self, questions, contents, tokenizer, max_length=128):
        self.questions = questions
        self.contents = contents
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        content = self.contents[idx]

        # 질문과 컨텐츠를 각각 토큰화
        question_encoding = self.tokenizer(
            question,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        content_encoding = self.tokenizer(
            content,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'question_input_ids': question_encoding['input_ids'].squeeze(),
            'question_attention_mask': question_encoding['attention_mask'].squeeze(),
            'content_input_ids': content_encoding['input_ids'].squeeze(),
            'content_attention_mask': content_encoding['attention_mask'].squeeze()
        }

# 모델 정의
class SimilarityModel(nn.Module):
    def __init__(self, model_name):
        super(SimilarityModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.fc = nn.Linear(self.model.config.hidden_size * 2, 1)

    def forward(self, question_input_ids, question_attention_mask, content_input_ids, content_attention_mask):
        # 질문 임베딩 추출
        question_output = self.model(
            input_ids=question_input_ids,
            attention_mask=question_attention_mask
        )
        question_embedding = question_output.pooler_output

        # 컨텐츠 임베딩 추출
        content_output = self.model(
            input_ids=content_input_ids,
            attention_mask=content_attention_mask
        )
        content_embedding = content_output.pooler_output

        # 질문과 컨텐츠 임베딩을 합쳐서 유사도 예측
        combined_embedding = torch.cat((question_embedding, content_embedding), dim=1)
        similarity_score = self.fc(combined_embedding)

        return similarity_score

# 학습 루프 정의
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Training", leave=False):
        # 입력 데이터 준비
        question_input_ids = batch['question_input_ids'].to(device)
        question_attention_mask = batch['question_attention_mask'].to(device)
        content_input_ids = batch['content_input_ids'].to(device)
        content_attention_mask = batch['content_attention_mask'].to(device)

        # 모델 예측 및 손실 계산
        optimizer.zero_grad()
        similarity_score = model(question_input_ids, question_attention_mask, content_input_ids, content_attention_mask)
        labels = torch.ones(similarity_score.size()).to(device)  # 긍정 쌍이므로 모든 레이블은 1
        loss = criterion(similarity_score, labels)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

# 하이퍼파라미터 및 데이터 로딩
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)

questions = data['question'].tolist()
contents = data['content'].tolist()
dataset = PositivePairDataset(questions, contents, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 모델, 옵티마이저, 손실 함수 설정
model = SimilarityModel(model_name).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

# 학습 실행
num_epochs = 2
for epoch in range(num_epochs):
    avg_loss = train(model, dataloader, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/2, Loss: 0.0012


Epoch 2/2, Loss: 0.0000


In [21]:
# 모델 저장
torch.save(model.state_dict(), "similarity_model.pth")
print("Model saved successfully.")

Model saved successfully.
